In [1]:
import sys; sys.path.insert(0, '..')
from database.dataset import *

from feature_extraction.feature_extraction import CandidatesFeatureExtraction, CandidatesFeatureExtractionMP
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from general_utils.plots import plot_bboxes_over_image, simple_im_show, simple_im_show2
from metrics.metrics import get_tp_fp_fn, quick_circle_comparison, circle_comparison


In [2]:
import radiomics

radiomics.logger.setLevel(logging.ERROR)


In [3]:
dehazing_params = {'omega': 0.9, 'window_size': 11, 'radius': 40, 'eps': 1e-5}

hough1_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                 'param1': 300, 'param2': 8,  'minRadius': 2, 'maxRadius': 20}

hough2_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                 'param1': 300, 'param2': 10,  'minRadius': 2, 'maxRadius': 20}

back_ext_radius = 50

erosion_iter = 20
erosion_size = 5


hd = HoughCalcificationDetection(dehazing_params, back_ext_radius,
                                 Path.cwd().parent / 'data/hough_img',    # "/media/vzalevskyi/DATA/ProjectsData/calc-det/no_dehaizing",
                                 hough1_params, hough2_params,
                                 erosion_iter=erosion_iter,
                                 erosion_size=erosion_size)


In [4]:
db = INBreast_Dataset(
        return_lesions_mask=True,
        level='image',
        max_lesion_diam_mm=1.0,
        extract_patches=True,
        extract_patches_method='all',  # 'centered'
        patch_size=256,
        stride=256,
        min_breast_fraction_roi=0.5,
        normalize=None,
        n_jobs=-1,
)

In [5]:
idx = 0
image = db[idx]['img']
image_id = db.df.iloc[idx].img_id
image_mask = db[idx]['lesion_mask']

Available feature groups are: `['firstorder', 'glcm', 'gldm', 'glrlm', 'glszm', 'ngtdm', 'shape', 'shape2D']`.


To include all features for a feature group use `[]` or `None` as the key in settings dict, like: `{'firstorder':[]}`, otherwise we can specify features using their names `{'firstorder':['10Percentile]}`

Image types specify for which images should these features be calculated for: original image, normalized, wavelet, etc.

https://pyradiomics.readthedocs.io/en/latest/customization.html


In [6]:
feature_types = {'firstorder':[], 'shape2D':[], 'glcm':[]}
image_types = {'Original':{}}

# FE class initialization
cfe  = CandidatesFeatureExtraction(feature_types, image_types, patch_size=30)
# cfe_mp  = CandidatesFeatureExtractionMP(feature_types, image_types, patch_size=30, n_jobs=6)

In [7]:
from scipy.stats import skew, kurtosis
epsillon = np.finfo(float).eps


def entropy_uniformity(image_patch):
    _, counts = np.unique(image_patch, return_counts=True)
    norm_counts = counts / counts.sum()
    uniformity = (norm_counts**2).sum()
    return -(norm_counts * np.log2(norm_counts + epsillon)).sum(), uniformity


def fof(image_patch):
    patch_size = image_patch.shape[0]*image_patch.shape[1]

    img_energy = (image_patch**2).sum()
    img_entropy, img_uniformity = entropy_uniformity(image_patch)
    img_min = image_patch.min()
    img_10th_perc = np.quantile(image_patch, q=0.1)
    img_90th_perc = np.quantile(image_patch, q=0.9)
    img_max = np.quantile(image_patch, q=0.1)
    img_mean = np.mean(image_patch)
    img_meadian = np.median(image_patch)
    img_inter_quartile_range = np.quantile(
        image_patch, q=0.75) - np.quantile(image_patch, q=0.25)
    img_range = img_max - img_min
    img_mean_abs_deviation = np.abs(image_patch - img_mean).sum()/patch_size

    robust_img = image_patch[(image_patch <= img_90th_perc) & (
        image_patch >= img_10th_perc)]
    img_robust_mean_abs_deviation = np.abs(
        robust_img - robust_img.mean()).sum()/len(robust_img)

    img_rms = np.sqrt((image_patch**2).sum()/patch_size)
    img_std = np.std(image_patch)
    img_skew = skew(image_patch.ravel())
    img_kurt = kurtosis(image_patch.ravel())

    return {'img_energy': img_energy,
            'img_entropy': img_entropy,
            'img_uniformity': img_uniformity,
            'img_min': img_min,
            'img_10th_perc': img_10th_perc,
            'img_90th_perc': img_90th_perc,
            'img_max': img_max,
            'img_mean': img_mean,
            'img_meadian': img_meadian,
            'img_inter_quartile_range': img_inter_quartile_range,
            'img_range': img_range,
            'img_mean_abs_deviation': img_mean_abs_deviation,
            'img_robust_mean_abs_deviation': img_robust_mean_abs_deviation,
            'img_rms': img_rms,
            'img_std': img_std,
            'img_skew': img_skew,
            'img_kurt': img_kurt}


In [8]:
h1_circles = hd.detect(
    image, image_id, load_processed_images=True, hough2=False)[0]

In [9]:
old_res = cfe.extract_features(h1_circles, image, image_mask, sample=None)
old_res = pd.DataFrame(old_res)
old_res

100%|██████████| 1490/1490 [00:01<00:00, 1044.42it/s]


,img_energy,img_entropy,img_uniformity,img_min,img_10th_perc,img_90th_perc,img_max,img_mean,img_meadian,img_inter_quartile_range,img_range,img_mean_abs_deviation,img_robust_mean_abs_deviation,img_rms,img_std,img_skew,img_kurt,coordinates,patch_mask_intersection
0,29751096,7.869762,0.005602,1355,1439.9,1895.1,1439.9,1564.797778,1503.0,89.00,84.9,122.914296,56.755556,181.815218,177.499606,1.944877,2.694606,"((1027, 1057), (2280, 2310))",0
1,28174928,8.035920,0.004452,1768,1886.9,2102.0,1886.9,2003.357778,2014.0,112.25,118.9,65.929931,46.879084,176.933534,81.702637,-0.457342,-0.288067,"((1257, 1287), (1590, 1620))",0
2,27668287,7.707045,0.005612,1724,1838.0,1998.0,1838.0,1922.150000,1925.0,80.00,114.0,48.390778,33.616773,175.335510,60.785952,-0.173033,-0.094602,"((862, 892), (1651, 1681))",0
3,29447735,7.512864,0.006346,1976,2082.9,2218.0,2082.9,2151.181111,2152.0,68.00,106.9,41.770612,29.433962,180.885891,52.042157,-0.156237,-0.062458,"((1050, 1080), (706, 736))",0
4,28796024,8.368521,0.003474,1760,1862.0,2164.1,1862.0,2003.406667,2000.0,177.25,102.0,94.309941,72.345837,178.873090,111.737366,0.168333,-0.900444,"((1056, 1086), (732, 762))",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,29770233,7.545373,0.006084,1437,1495.9,1642.1,1495.9,1572.667778,1573.5,85.25,58.9,45.689978,33.930556,181.873683,55.217707,-0.100077,-0.630051,"((517, 547), (1951, 1981))",0
1486,29732686,7.269818,0.007686,1340,1398.0,1508.0,1398.0,1451.508889,1449.0,55.00,58.0,34.362657,23.342801,181.758955,44.437083,0.581106,0.847189,"((674, 704), (701, 731))",0
1487,29000930,7.302889,0.007479,1226,1286.9,1401.0,1286.9,1346.015556,1348.0,58.25,60.9,35.277536,24.366034,179.508372,45.001454,0.124306,0.103896,"((704, 734), (3094, 3124))",0
1488,29992556,8.229962,0.003835,1477,1558.0,1823.2,1558.0,1687.280000,1684.5,145.25,81.0,82.226222,59.641585,182.551533,101.053789,0.323341,-0.403189,"((717, 747), (2179, 2209))",0


In [10]:
old_res

""


In [42]:
fof(img_patch[0])

{'img_energy': 27668287,
 'img_entropy': 7.707044921593821,
 'img_uniformity': 0.005612345679012346,
 'img_min': 1724,
 'img_10th_perc': 1838.0,
 'img_90th_perc': 1998.0,
 'img_max': 1838.0,
 'img_mean': 1922.15,
 'img_meadian': 1925.0,
 'img_inter_quartile_range': 80.0,
 'img_range': 114.0,
 'img_mean_abs_deviation': 48.39077777777777,
 'img_robust_mean_abs_deviation': 33.61677274457702,
 'img_rms': 175.33551012590436,
 'img_std': 60.78595186755279,
 'img_skew': -0.17303324642144266,
 'img_kurt': -0.09460179748800845}

In [31]:


def detect(img_patch):
    sigma_min = 1
    sigma_max = 7
    return feature.multiscale_basic_features(img_patch, intensity=True, edges=True, texture=True,
                                             sigma_min=sigma_min, sigma_max=sigma_max, num_workers=6)
    

In [34]:
detect(image[100:130, 100:130]).shape

(30, 30, 12)

In [28]:
res_mp = cfe.extract_features(h1_circles, image, image_mask, sample=None)

  2%|▏         | 28/1490 [00:12<11:01,  2.21it/s]


KeyboardInterrupt: 

In [ ]:
res = cfe.extract_features(candidates, image, image_mask, sample=70)

In [15]:
# check that MP and regular classes return the same
assert np.all((pd.DataFrame(res) == pd.DataFrame(res_mp)).values)

In [16]:
pd.DataFrame(res)

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,coordinates,patch_mask_intersection
0,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
1,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
2,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
3,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
4,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
96,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
97,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
98,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
